In [1]:
# for everything

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# for random forest

from sklearn.ensemble import RandomForestRegressor

In [2]:
rawdat = pd.read_csv('../1500_samples.csv')
len(rawdat)


dat = rawdat.copy()
dat.tail()


dat = dat[(dat['n_hill1'] == 1000)]

train_dat = dat.sample(frac = 0.7, replace = False, random_state = 0)

test_dat = dat.drop(train_dat.index)

train_X = train_dat[['richness', 'abundance']]
train_Y = train_dat[['mean', 'sd']]


test_X = test_dat[['richness', 'abundance']]
test_Y = test_dat[['mean', 'sd']]

In [3]:
test_results = {}

rf_mean = RandomForestRegressor()
rf_mean.fit(train_X, train_Y['mean'])
test_results['rf_mean_r2'] = rf_mean.score(test_X, test_Y['mean'])

rf_sd = RandomForestRegressor()
rf_sd.fit(train_X, train_Y['sd'])
test_results['rf_sd_r2'] = rf_sd.score(test_X, test_Y['sd'])

In [4]:
test_results

{'rf_mean_r2': 0.9987369882877134, 'rf_sd_r2': 0.9959755988965413}

In [5]:
rf_test_preds = test_X.copy()
rf_test_preds['rf_sd'] = rf_sd.predict(test_X)
rf_test_preds['rf_mean'] = rf_mean.predict(test_X)

In [6]:
rf_test_preds.tail()

,richness,abundance,rf_sd,rf_mean
1452,196,12312,7.869987,108.736385
1454,197,3982,9.824170,86.968656
1455,197,7744,8.749775,101.131527
1462,198,3541,10.228613,86.110921
1471,200,11334,8.064898,107.181608


In [7]:
# rf_test_preds.to_csv("random_forest_predictions.csv")

original_samples = rawdat = pd.read_csv('../1500_samples_raw.csv')
original_samples = original_samples[["richness", "abundance", "hill1"]]


,richness,abundance,hill1
869000,196,12312,110.717455
869001,196,12312,94.882695
869002,196,12312,94.948767
869003,196,12312,102.913038
869004,196,12312,102.905682
...,...,...,...
869995,196,12312,104.679975
869996,196,12312,110.449493
869997,196,12312,102.124657
869998,196,12312,110.614897


In [184]:
def draw_from_normal(mean, sd):
    draws = np.random.default_rng().normal(mean, sd, 1000)
    return(draws)
    
def sample_a_row(a_row_of_x, orig_samples):
    s = a_row_of_x[1]
    n = a_row_of_x[2]
    draws = sorted(draw_from_normal(a_row_of_x[4], a_row_of_x[3]))
    orig_samples_matched = orig_samples.loc[(orig_samples['richness'] == s) & (orig_samples['abundance'] ==n)]
    x_samples = {"sampled_hill": draws,
                "richness": np.repeat(s, 1000),
                "abundance": np.repeat(n, 1000)}
    x_samples = pd.DataFrame(data=x_samples)
    x_samples["hill1"] = (orig_samples_matched[['hill1']].values)
    x_samples['hill1'] = sorted(x_samples['hill1'])

    return(x_samples)

In [89]:
short_test_preds = rf_test_preds[:5]

short_test_preds

,richness,abundance,rf_sd,rf_mean
0,3,3606,0.420373,2.357711
3,3,8083,0.415837,2.356915
7,3,11700,0.424099,2.345248
11,3,18166,0.427433,2.336939
23,5,8307,0.629639,3.646632


In [185]:
sampled_df = pd.DataFrame()


for row in rf_test_preds.itertuples():
    sampled_df = pd.concat([sampled_df, (sample_a_row(row, orig_samples = original_samples.copy()))])
    


In [186]:


sampled_df.head()

,sampled_hill,richness,abundance,hill1
0,1.085190,3,3606,1.148415
1,1.155295,3,3606,1.154451
2,1.187851,3,3606,1.170528
3,1.226245,3,3606,1.176418
4,1.282522,3,3606,1.182063


In [183]:
type(sampled_df[['hill1']])

pandas.core.frame.DataFrame

In [187]:
sampled_df.to_csv("rf_and_fs_samples.csv")